# Classification

**COMP4670/8600 &mdash; Statistical Machine Learning &mdash; Week 4**

In this lab we will build, train, and test a logistic regression classifier.

### Assumed knowledge:

- Optimisation in Python (week 1 lab)
- Regression (week 2 lab)
- Binary classification with logistic regression (week 3 lectures)

### After this lab, you should be comfortable with:

- Implementing logistic regression
- Practical binary classification problems

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt

%matplotlib inline

## The data set

We will be working with the census-income dataset, which shows income levels for people in the 1994 US Census. We will predict whether a person has $\leq \$50000$ or $> \$50000$ income per year.

The data are included with this notebook as `04-dataset.tsv`, a textfile where in each row of data, the individual entries are delimited by tab characters. Download the data from the [course website](https://machlearn.gitlab.io/sml2019/tutorials/04-dataset.tsv)
Load the data into a NumPy array called `data` using `numpy.genfromtxt`:

```python
    numpy.genfromtxt(filename)
```

The column names are given in the variable `columns` below.
The `income` column are the targets, and the other columns will form our data used to try and guess the `income`

In [2]:
columns = ['income', 'age', 'education', 'private-work', 'married', 'capital-gain', 'capital-loss', 'hours-per-week']

In [3]:
data = np.genfromtxt("04-dataset.tsv")

In [4]:
data[:,0]

array([0., 0., 0., ..., 0., 0., 1.])

## Recap - Binary classification

The idea behind this lab is that for each person, we want to
try and predict if their income is above the threshold of $\$50,000$ or not,
based on a series of other data about their person: `age, education,...`.

As per usual, for the $n^\text{th}$ row, the first entry is the target $t_n$, and the rest
forms the data vector $\mathbf{x}_n$

We have two classes, $C_1$ representing the class of $ <\$ 50,000$, which corresponds to
a target of $t_n = 0$, and $C_2$, representing the class of $ >\$50,000$, corresponding to
a target of $t_n = 1$. Our objective is to learn a discriminative function $f_{\mathbf{w}}(\mathbf{x})$,
parametrised by a weight vector $\mathbf{w}$ that
predicts which income class the person is in, based on the data given.

We assume that each piece of information $(t_n, \mathbf{x}_n)$ is i.i.d, and
that there is some hidden probability distribution from which these target/data points are drawn.
We will construct a likelihood function that indicates ''what is the likelihood that of this particular
weight vector $\mathbf{w}$, given that we have observed the training data $(t_n, \mathbf{x}_n)$?''.
$\{t_1, t_2, \ldots \}$ would be generated?"

## Recap - Feature map, basis function

Now some classes are not linearly seperable (we cannot draw a line such that all of one class is on one side,
and all of the other class is on the other side). But by applying a fixed non-linear 
transformation to the inputs $\mathbf{x}_i$ first, the result is usually linearly, for a suitable choice
of transformation $\phi$. (See week 3, pg 342 of the lecture slides).

We let
$$
\mathbf{\phi}_n := \phi(\mathbf{x}_n)
$$

The result is linear in $\phi$ but not in $\mathbf{x}$, so we work in feature space rather than
input space.
For the case of two classes, we could guess that the target is a linear combination of the features,
$$
\hat{t}_n = \mathbf{w}^T \mathbf{\phi}_n
$$
but $\mathbf{w}^T \mathbf{\phi}_n$ is a real number, and we want $\hat{t}_n \in \{0,1\}$.
We could threshold the result,
$$
\hat{t}_n =
\begin{cases}
1 & \mathbf{w}^T \mathbf{\phi}_n \geq 0 \\
0 & \mathbf{w}^T \mathbf{\phi}_n < 0
\end{cases}
$$
but the discontinuity makes it impossible to define a sensible gradient. 

## Recap - Logistic Regression

(We assume that the classes are already linearly seperable, and use our input space as our feature space.
We also assume the data is i.i.d).

Instead of using a hard threshold like above, in logistic regression
we can use the sigmoid function $\sigma(a)$
$$
\sigma(a) := \frac{1}{1 + e^{-a}}
$$
which has the intended effect of "squishing" the real line to the interval $[0,1]$.
This gives a smooth version of the threshold function above, that we can differentiate.
The numbers it returns can be interpreted as a probability of the estimated target $\hat{t}$ belonging
to a class $C_i$ given the element $\phi$ of feature space. In the case of two classes, we define
\begin{align}
p(C_1 | \phi ) &:= \sigma (\mathbf{w}^T \phi) \\
p(C_2 | \phi ) &:= 1 - p(C_1 | \phi)
\end{align}



The likelihood function $p(\mathbf{t} | \mathbf{w}, \mathbf{x})$ is what we want to maximise as a function
of $\mathbf{w}$. Since $\mathbf{x}$ is fixed, we usually write the likelihood function as $p(\mathbf{t} | \mathbf{w})$.

$$
\begin{align}
p(\mathbf{t} | \mathbf{w})
&= \prod_{i=1}^n p(t_i | \mathbf{w}) \\
&= \prod_{i=1}^n 
\begin{cases}
p(C_1 | \phi_i) & t_i = 1 \\
p(C_2 | \phi_i) & t_i = 0
\end{cases}
\end{align}
$$
Note that
$$
\begin{cases}
 y_n & t_i = 1 \\
1 - y_n & t_i = 0
\end{cases}
= y_n^{t_n} (1-y_n)^{1-t_n}
$$
as if $t_n = 1$, then $y_n^1 (1-y_n)^{1-1} = y_n$ and if $t_n = 0$ then $y_n^0 (1-y_n)^{1-0} = 1-y_n$.
This is why we use the strange encoding of $t_i=0$ corresponds to $C_2$ and $t_i=1$ corresponds to $C_1$.
Hence, our likelihood function is 
$$
p(\mathbf{t} | \mathbf{w}) = \prod_{n=1}^N y_n^{t_n} (1-y_n)^{1-t_n}, \quad y_n = \sigma(\mathbf{w}^T \phi_n)
$$
Because
$$
p(t_n | \mathbf{w}) = y_n^{t_n} (1-y_n)^{1-t_n}
$$
This function is quite unpleasant to try and differentiate, but we note that $p(\mathbf{t} | \mathbf{w})$
is maximised when $\log p(\mathbf{t} | \mathbf{w})$ is maximised.
\begin{align}
\log p(\mathbf{t} | \mathbf{w}) 
&= \log \prod_{i=1}^N y_n^{t_n} (1-y_n)^{1-t_n} \\
&= \sum_{i=1}^N \log \left( y_n^{t_n} (1-y_n)^{1-t_n} \right) \\
&= \sum_{i=1}^N \left( t_n \log y_n +  (1-t_n) \log (1-y_n) \right)
\end{align}
Which is maximised when $- \log p(\mathbf{t} | \mathbf{w})$ is minimised, giving us our error function.
$$
E(\mathbf{w}) := - \sum_{i=1}^N \left( t_n \log y_n +  (1-t_n) \log (1-y_n) \right)
$$
We can then take the derivative of this. As an exercise, you should do this (using the identity $\sigma'(a) = \sigma(a) \left( 1- \sigma(a) \right)$ to simplify).
$$
\nabla_\mathbf{w} E(\mathbf{w}) = \sum_{i=1}^N (y_n - t_n) \phi_n
$$
which you will note doesn't have any sigmoid functions.

(We also usually divide the error by the number of data points, to obtain the average error. The error
shouldn't get 10 times as large just because there is more data avaliable, so we should divide by the
number of error points to reflect that.)

## Recap - $L_2$ regularisation, Gaussian prior

To help avoid overfitting, we can add a penalty term to the cost function of the form 
$\frac{\lambda}{2} ||\mathbf{w}||^2$. By tweaking the value of $\lambda$, we can indicate how
much to penalise large terms in the weight vector $\mathbf{w}$. Don't forget to take the regularistion term into
account when you compute the corresponding gradient $\nabla_\mathbf{w} E(\mathbf{w})$.


## Explain logistic regression (10 minutes)

Find a partner in your lab (or groups of 3). Take turns to explain the topics above to each other, without referring to the lab sheet. Be as precise as possible, by writing down the relevant equations.


## Classification with logistic regression

Implement binary classification using logistic regression and $L_2$ regularisation. Make sure you write good quality code with comments and docstrings where appropriate.

Use ```scipy.optimize.fmin_bfgs``` to optimise your cost function. ```fmin_bfgs``` takes as arguments the cost function to be optimised, and a tuple of extra arguments to the cost function:

```python
    scipy.optimise.fmin_bfgs(cost_function, initial_guess, args=())
```

By following equations in lectures, implement three functions:

- `grad(w, X, t, a)`, which calculates the gradient of the cost function,
- `train(X, t, a)`, which returns the maximum likelihood weight vector, and
- `test(w, X)`, which returns predicted class probabilities,

where 
* $w$ is a weight vector, 
* $X$ is a matrix of examples, 
* $t$ is a vector of labels/targets, 
* $a$ is the regularisation weight. 

(We would use $\lambda$ for the regularisation term, but `a` is easier to type than `lambda`, and
`lambda` is a reserved keyword in python, for lambda functions).

See below for expected usage.

We add an extra column of ones to represent the bias term.

## Note

* You should use 80% of the data as your training set, and 20% of the data as your test set.
* You also may want to normalise the data before hand. If the magnitude of $\mathbf{w}^T \phi_n$
is very large, the gradient of $\sigma(\mathbf{w}^T \phi_n)$ will be very near zero, which can
cause convergence issues during numerical minimisation. If each element in a particular column is
multiplied by a scalar (say, all elements of the `age` column) then the result is essentially the same
as stretching the space in which the data lives. The model will also be proportionally stretched,
but will not fundamentally change the behaviour. So by normalising each column, we can avoid
issues related to numerical convergence.

In [5]:
# Normalize data
def normalize(x, amin, amax):
    m = np.mean(x)
    return (x-m)/(amax-amin)
vnorm = np.vectorize(normalize)

for i in range(1,data.shape[1]):
    col = data[:,i]
    amin = np.amin(col)
    amax = np.amax(col)
    data[:,i] = normalize(data[:,i], amin, amax)


array([[ 0.        ,  0.00573087,  0.19462138, ...,  0.01096362,
        -0.0200422 , -0.00446384],
       [ 0.        ,  0.1564158 ,  0.19462138, ..., -0.0107766 ,
        -0.0200422 , -0.27997404],
       [ 0.        , -0.00796776, -0.07204529, ..., -0.0107766 ,
        -0.0200422 , -0.00446384],
       ...,
       [ 0.        ,  0.26600484, -0.07204529, ..., -0.0107766 ,
        -0.0200422 , -0.00446384],
       [ 0.        , -0.22714585, -0.07204529, ..., -0.0107766 ,
        -0.0200422 , -0.20854547],
       [ 1.        ,  0.18381306, -0.07204529, ...,  0.13946491,
        -0.0200422 , -0.00446384]])

In [6]:
data

array([[ 0.        ,  0.00573087,  0.19462138, ...,  0.01096362,
        -0.0200422 , -0.00446384],
       [ 0.        ,  0.1564158 ,  0.19462138, ..., -0.0107766 ,
        -0.0200422 , -0.27997404],
       [ 0.        , -0.00796776, -0.07204529, ..., -0.0107766 ,
        -0.0200422 , -0.00446384],
       ...,
       [ 0.        ,  0.26600484, -0.07204529, ..., -0.0107766 ,
        -0.0200422 , -0.00446384],
       [ 0.        , -0.22714585, -0.07204529, ..., -0.0107766 ,
        -0.0200422 , -0.20854547],
       [ 1.        ,  0.18381306, -0.07204529, ...,  0.13946491,
        -0.0200422 , -0.00446384]])

In [7]:
# v = np.array([1,2,3,4,5])
# amin = np.amin(v)
# amax = np.amax(v)
# vnorm(v,amin,amax)

In [8]:
assert data.shape[1] == 8, 'you already ran this cell!'
data = np.concatenate([data, np.ones((data.shape[0], 1))], axis=1)  # add a column of ones
data

array([[ 0.        ,  0.00573087,  0.19462138, ..., -0.0200422 ,
        -0.00446384,  1.        ],
       [ 0.        ,  0.1564158 ,  0.19462138, ..., -0.0200422 ,
        -0.27997404,  1.        ],
       [ 0.        , -0.00796776, -0.07204529, ..., -0.0200422 ,
        -0.00446384,  1.        ],
       ...,
       [ 0.        ,  0.26600484, -0.07204529, ..., -0.0200422 ,
        -0.00446384,  1.        ],
       [ 0.        , -0.22714585, -0.07204529, ..., -0.0200422 ,
        -0.20854547,  1.        ],
       [ 1.        ,  0.18381306, -0.07204529, ..., -0.0200422 ,
        -0.00446384,  1.        ]])

In [9]:
data

array([[ 0.        ,  0.00573087,  0.19462138, ..., -0.0200422 ,
        -0.00446384,  1.        ],
       [ 0.        ,  0.1564158 ,  0.19462138, ..., -0.0200422 ,
        -0.27997404,  1.        ],
       [ 0.        , -0.00796776, -0.07204529, ..., -0.0200422 ,
        -0.00446384,  1.        ],
       ...,
       [ 0.        ,  0.26600484, -0.07204529, ..., -0.0200422 ,
        -0.00446384,  1.        ],
       [ 0.        , -0.22714585, -0.07204529, ..., -0.0200422 ,
        -0.20854547,  1.        ],
       [ 1.        ,  0.18381306, -0.07204529, ..., -0.0200422 ,
        -0.00446384,  1.        ]])

In [10]:
# Split data to training and test sets
split_point = int(data.shape[0]*0.8)
np.random.shuffle(data)
train_data = data[:split_point,:]
test_data = data[split_point:,:]

print(train_data.shape)
print(test_data.shape)

#split train and test data into feature and target
train_feature = train_data[:,1:]
train_target = train_data[:,0]
print(train_feature)
print(train_target)

test_feature = test_data[:,1:]
test_target = test_data[:,0]
print(test_feature.shape)
print(test_target.shape)


(26048, 9)
(6513, 9)
[[ 0.10162128 -0.00537862  0.30296981 ... -0.0200422  -0.00446384
   1.        ]
 [-0.1175568  -0.07204529  0.30296981 ... -0.0200422   0.03635249
   1.        ]
 [-0.1175568   0.12795471  0.30296981 ... -0.0200422  -0.00446384
   1.        ]
 ...
 [-0.09015954 -0.00537862  0.30296981 ... -0.0200422  -0.00446384
   1.        ]
 [-0.13125544  0.26128804  0.30296981 ... -0.0200422  -0.00446384
   1.        ]
 [ 0.12901854 -0.00537862  0.30296981 ... -0.0200422  -0.20854547
   1.        ]]
[0. 0. 0. ... 0. 1. 0.]
(6513, 8)
(6513,)


In [23]:
# sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
asigmoid = np.vectorize(sigmoid)

# Negative Log Likelihood Error function
def E(w,X,t,a):
    y = sigmoid(X@w)
    return (- np.dot(t, np.log(y))
            - np.dot((np.ones(t.shape[0]) - t), np.log((np.ones(t.shape[0]) - y)))
            + (a*0.5) * ((np.linalg.norm(w))**2) )

# Derivative of Error function
def dE(w,X,t,a):
    """
    X - feature matrix
    w - weights
    t - target
    a - regularization multiplier
    """
    y = sigmoid(X@w)
    return ((X.T @ (y - t)) + a*w)

# Gradient descent training
def train(w,X,t,a,n,d):
    """
    a - regularization multiplier
    """
    wopt = opt.fmin_bfgs(E, w, fprime=dE, 
                        args=(X,t,a))
    return wopt

def test(w,X):
    """
    w - optimized weight
    """
    return sigmoid(X@w)

In [18]:
wtest = np.array([3,4])
Xtest = np.array([[1,4],
            [2,5],
            [3,6]])
ttest = np.array([0,1,1])
atest = 0.1

error = E(wtest,Xtest,ttest,atest)
print("error:",error)
de = dE(wtest,Xtest,ttest,atest)
print(de)

error: 20.2499999979572
[1.29999999 4.39999998]


In [22]:
X = train_feature
n = X.shape[0]
d = X.shape[1]
w = 10*np.random.rand(X.shape[1])
t = train_target
a = 0.1 #regularization multiplier

w_star = train(w,X,t,a,n,d)
print(w_star)

         Current function value: 8903.220081
         Iterations: 22
         Function evaluations: 107
         Gradient evaluations: 95
[ 2.13485084  5.43659509  0.19849671  2.37899843 28.05825424  2.91567622
  3.29821683 -1.77282342]


## Performance measure

There are many ways to compute the performance of a binary classifier. The key concept is the idea of a confusion matrix:

|     &nbsp;         | &nbsp;  | Label | &nbsp;  |
|:-------------:|:--:|:-----:|:--:|
|     &nbsp;         |  &nbsp;  |  0    | 1  |
|**Prediction**| 0  |    TN | FN |
|      &nbsp;        | 1  |    FP | TP |

where
* TP - true positive
* FP - false positive
* FN - false negative
* TN - true negative

Implement three functions:

- `confusion_matrix(y_true, y_pred)`, which returns the confusion matrix as a list of lists given a list of true labels and a list of predicted labels;
- `accuracy(cm)`, which takes a confusion matrix and returns the accuracy; and
- `balanced_accuracy(cm)`, which takes a confusion matrix and returns the balanced accuracy.

The accuracy is defined as $\frac{TP + TN}{n}$, where $n$ is the total number of examples. The balanced accuracy is defined as $\frac{1}{2}\left(\frac{TP}{P} + \frac{TN}{N}\right)$, where $T$ and $N$ are the total number of positive and negative examples respectively.

In [34]:
def confusion_matrix(y_true, y_pred):
    conf = np.zeros((2,2))
    for i in range(y_pred.shape[0]):
        if y_true[i] == 0:
            if y_pred[i] < 0.5:
                conf[0,0] += 1 #TN
            else:
                conf[1,0] += 1 #FP
        else:
            if y_pred[i] <= 0.5:
                conf[1,0] += 1 #FN
            else:
                conf[1,1] += 1 #TP
    return conf

def accuracy(conf):
    t = conf[0,0] + conf[1,1]
    f = conf[0,1] + conf[1,0]
    return t/(t+f)

def balanced_accuracy(conf):
    tp = conf[1,1]
    tn = conf[0,0]
    p = conf[1,0] + conf[1,1]
    n = conf[0,0] + conf[0,1]
    return 0.5*(tp/p + tn/n)

test_preds = test(w_star, test_feature)
conf = confusion_matrix(test_target, test_preds)
acc = accuracy(conf)
bacc = balanced_accuracy(conf)
print(conf)
print("accuracy:",acc)
print("balanced accuracy:", bacc)


[[4616.    0.]
 [1022.  875.]]
accuracy: 0.8430830646399509
balanced accuracy: 0.7306273062730627


## Accuracy vs balanced accuracy

What is the purpose of balanced accuracy? When might you prefer it to accuracy?

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

## Putting everything together

Consider the following code, which trains on all the examples, predicts on the training set, and then computes the accuracy and balanced accuracy. Discuss the results.

In [ ]:
# replace this with your solution, add and remove code and markdown cells as appropriate

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

## Looking back at the prediction task

Based on your results, what feature of the dataset is most useful for determining the income level? What feature is least useful? Why?

In [ ]:
# replace this with your solution, add and remove code and markdown cells as appropriate

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>